In [28]:
import snappy
import ast
import csv
import pickle
import random
import time 
from ast import literal_eval
from itertools import product

Let's import simplified diagrams of the census knots, which have not be determined to be strongly quasipositive () for those the Seifert genus already gives us the slice genus) 

In [29]:
# Lets get the
braid_words_file=open('/Users/leomousseau/Documents/census-seminar/seminar-2021/braid_words.txt','r')
braid_words_string=braid_words_file.read()
braid_words_list=ast.literal_eval(braid_words_string)
braid_dict=dict(braid_words_list)

# 3 genus
genus=[]
with open('/Users/leomousseau/Documents/census-seminar/seminar-2021/3_genus,top_smooth_4_genus,unkotting_number,double_slice_genus/3genus.csv','r') as inp:
    reader=csv.reader(inp)
    for row in reader:
        genus.append(row)
del genus[0]
for i in range(len(genus)):
    genus[i][1]=int(genus[i][1])
genus_dict=dict(genus)

pd_codes_not_SQP=[]  # These are all knots who have not been determined to be SQP
with open('/Users/leomousseau/Desktop/Bachelor code census/pd_codes_not_SQP','r') as inp:
    reader=csv.reader(inp)
    for row in reader:
        pd_codes_not_SQP.append(row)
for k in pd_codes_not_SQP:
    k[1]=literal_eval(k[1])


In [30]:
name_slice_knotinfo=[]
with open('/Users/leomousseau/Desktop/Bachelor code census/correctname_slice_knotinfo','r') as inp:
    reader=csv.reader(inp)
    for row in reader:
        name_slice_knotinfo.append([row[0],int(row[1])])
slice_genus_knotinfo=dict(name_slice_knotinfo)
        
name_top_knotinfo=[]
with open('/Users/leomousseau/Desktop/Bachelor code census/correctname_top_knotinfo','r') as inp:
    reader=csv.reader(inp)
    for row in reader:
        try:
            name_top_knotinfo.append([row[0],int(row[1])])
        except:
            name_top_knotinfo.append([row[0],2]) # those are all (1,2) and we bound them by 2 from above for our purposes later
topological_genus_knotinfo=dict(name_top_knotinfo)

In [31]:
len(pd_codes_not_SQP)

497

In [32]:
# lets remove the low crossing knots, since they are all prime (since they are hyperbolic) and therefore
# their slice genera can be obtained from KnotInfo
pd_codes=[]
for k in pd_codes_not_SQP:
    if len(k[1])<13:
        None
    else:
        pd_codes.append(k)


In [33]:
len(pd_codes)

409

In [34]:
def cc(pd,i):
    '''
    Changes the i-th crossing.
    '''
    PD_code=pd.copy()
    (a,b,c,d)=PD_code[i]
    PD_code[i]=(d,a,b,c)
    return PD_code

def cc(pd2,i):
    pd=pd2.copy()
    if pos_crossing(pd,i):
        [a,b,c,d]=pd[i]
        pd[i]=[d,a,b,c]
    else:
        [a,b,c,d]=pd[i]
        pd[i]=[b,c,d,a]
    return(pd)

def pos_crossing(pd,i):
    '''
    True if the i-th crossing is positive, False else
    '''
    n=max([max(k) for k in pd])
    (a,b,c,d)=pd[i]
    if d==n:
        if b==0:
            return(True)
    if b==n:
        if d==0:
            return(False)
    if d<b:
        return(True)
    return(False)



In [35]:
def crossing_changer(pd):
    '''
    input: A pd-code
    output: list of pd codes corresponding crossing changes in pd
    '''
    l=[]
    for i in range(len(pd)):
        new_pd=cc(pd,i)
        l.append([new_pd,[i,'cc']])
    return(l)


def two_crossing_changer(pd):
    '''
    input: pd code
    output: returns list of pd codes corresponding to changement of a positive and a negative crossing
    '''
    l=[]
    for i in range(len(pd)):
        if pos_crossing(pd,i):
            for j in range(len(pd)):
                if not pos_crossing(pd,j) and i!=j:
                    l.append([cc(cc(pd,i),j),[(i,j),'cc+-']])
    return(l)

In [36]:
def upper_bounds(pd4,counter=0,change_sequence=[]):
    '''
    input: A PD-code
    output: pd code, with list of operations done to obtain it
    '''
    pd=pd4.copy()
    counter+=1
    new_pds1=two_crossing_changer(pd)+crossing_changer(pd)#+resoluter(pd)
    new_pds2=[]
    for k in range(len(new_pds1)):
        t=new_pds1[k]         # t=[pd,(i,j),'cc+-''] or t=[pd,i,'cc']
        pd_new=t[0]
        K=snappy.Link(pd_new)
        K.simplify('level')
        simplified_pd=K.PD_code()
        new_pds2.append([simplified_pd,len(simplified_pd),t[1]]) 
        
    n=min([j[1] for j in new_pds2])        
    if n<13:  # we have reached a low crossing knot
        G=[]
        for [new_pd,c,change] in new_pds2:
            if c<13:
                
                (g_slice,g_top)=low_crossing_genus(new_pd)
                G.append([new_pd,(g_slice,g_top),change])



        slice_min=min([k[1][0] for k in G])
        top_min=min([k[1][1] for k in G])
 
        best_slice=[]
        best_top=[]
        for k in G:
            if k[1][0]==slice_min:
                best_slice.append(k)
            if k[1][1]==top_min:
                best_top.append(k)     

        if slice_min==top_min:
            
            [best_pd,(g1,g2),best_change]=best_slice[0] #or random
            change_sequence.append([pd,best_change])
            slice_bound=counter+g1
            top_bound=counter+g2
            return([(slice_bound,top_bound),change_sequence])

        else:
    
            [best_pd_slice,(g1_slice,g2_slice),best_change_slice]=best_slice[0] #or random
            [best_pd_top,(g1_top,g2_top),best_change_top]=best_top[0]
            change_sequence.append([pd,[best_change_slice,best_change_top]])
            slice_bound=counter+g1_slice
            top_bound=counter+g2_top
            return([(slice_bound,top_bound),change_sequence])


    else:    

        best_ones=[] 
        for j in new_pds2:
            if j[1]==n:
                best_ones.append([j[0],j[2]])  
        [best_pd,best_change]=best_ones[0] #or random here
        change_sequence.append([pd,best_change])
        return(upper_bounds(best_pd,counter,change_sequence))

In [37]:
def prime_decomposition(K,decomp=[]): # doesnt work perfectly since the implemented function deconnect sum is not perfec
    deconnect=K.deconnect_sum()
    if len(deconnect)==1: #i.e. K is prime
        decomp.append(K)
        return(decomp)
 
    for L in deconnect:
        decomp=decomp+prime_decomposition(L,[])
    
    return(decomp)

def low_crossing_genus(pd):
    '''
    input: pd code
    output: bound on both slice genera
    '''
    K=snappy.Link(pd)
    M=K.exterior()
    if len(M.fundamental_group().generators())==1 and M.fundamental_group().relators()==[]:
        return((0,0))
    name='undefined'
    m=0
    if name=='undefined' and m<100:
        if M.identify()==[]:
            M.randomize()
        else:
            name=M.identify()[0].name()
        m=m+1
    if name!='undefined':
        slice_genus=slice_genus_knotinfo[name]
        topological_genus=topological_genus_knotinfo[name]
        slice_bound=slice_genus
        top_bound=topological_genus
    else: #the knot is composite
        prime_decomp=prime_decomposition(K)
        slice_sum=0
        top_sum=0
        for L in prime_decomp:
            M=L.exterior()
            name='undefined'
            m=0
            if name=='undefined' and m<100:
                if M.identify()==[]:
                    M.randomize()
                else:
                    name=M.identify()[0].name()
                m=m+1
            try:
                slice_sum+=slice_genus_knotinfo[name]
                top_sum+=topological_genus_knotinfo[name]
            except:
               # its probably torus!
                #print('TORUS check')
                [x,y]=identify_torus(L.PD_code())
                if x=='True':
                    (p,q)=y
                    slice_sum+=(1/2)*(p-1)*(q-1)
                    top_sum+=(1/2)*(p-1)*(q-1)
                else:
                    print(L.PD_code(),'ERROR, it is not identifiable and not torus')
                    slice_sum+=5 #'largest slice genus among low crossing knots'
                    top_sum+=5 #'largest slice genus among low crossing knots'
            
        slice_bound=slice_sum
        top_bound=top_sum
        
    return(slice_bound,top_bound)    

In [38]:
def identify_torus(pd):
    '''
    input: a knot
    output: if the output is (p,q), then the knot is the p,q torus knot. If the return is false, then it is not
    '''
    K=snappy.Link(pd)
    if len(K.deconnect_sum())!=1:
        return(['False','non-prime!'])
    #therefore K is prime, and determined by its knot group
    g=K.exterior().fundamental_group()
    #print(g)
    if g.generators()==['a', 'b']:
        if len(g.relators())==1:
            word=g.relators()[0]
            counter=0
            for i in range(len(word)):
                if word[i]=='a' or word[i]=='A':
                    if i!=len(word)-1:
                        if word[i+1]=='b' or word[i+1]=='B':
                            counter=counter+1
            if counter==1: #its torus
                p=word.count('a')
                q=word.count('b')
                return(['True',(p,q)])
            
            if counter!=1:
                return(['False','relator word not right'])
        else:
            return(['False','relator'])
    else:
        return(['False','generators'])

In [39]:
n=0

for k in pd_codes:
    n=n+1
    [(g1,g2),sequence]=upper_bounds(k[1],0,[])
    print((g1,g2),snappy.Link(k[1]).knot_floer_homology()['tau'],'number:',n)
    l.append([k[0],(g1,g2),sequence])

(1, 1) 1 number: 1
(1, 1) 0 number: 2
(1, 1) 1 number: 3
(1, 1) 1 number: 4
(1, 1) 1 number: 5
(1, 1) 0 number: 6
(1, 1) 0 number: 7
(1, 1) 0 number: 8
(1, 1) 1 number: 9
(2, 2) 1 number: 10
(2, 2) 1 number: 11
(2, 2) -1 number: 12
(1, 1) 1 number: 13
(2, 2) 2 number: 14
(3, 3) 2 number: 15
(2, 2) 1 number: 16
(3, 3) -2 number: 17
(2, 2) -2 number: 18
(1, 1) 0 number: 19
(1, 1) -1 number: 20
(1, 1) 0 number: 21
(1, 1) 0 number: 22
(1, 1) 0 number: 23
(1, 1) -1 number: 24
(1, 1) 0 number: 25
(1, 1) 0 number: 26
(4, 4) -4 number: 27
(1, 1) 0 number: 28
(1, 1) 1 number: 29
(1, 1) 0 number: 30
(2, 2) -1 number: 31
(1, 1) 0 number: 32
(1, 1) 1 number: 33
(1, 1) 1 number: 34
(1, 1) 0 number: 35
(1, 1) 0 number: 36
(1, 1) 1 number: 37
(2, 2) 1 number: 38
(3, 3) 2 number: 39
(1, 1) 0 number: 40
(1, 1) 0 number: 41
(1, 1) 1 number: 42
(1, 1) -1 number: 43
(1, 1) 0 number: 44
(3, 3) -3 number: 45
(3, 3) 2 number: 46
(2, 2) 2 number: 47
(5, 5) -5 number: 48
(5, 5) -4 number: 49
(1, 1) 0 number: 5

(1, 1) 0 number: 389
(2, 2) 1 number: 390
(1, 1) 0 number: 391
(1, 1) -1 number: 392
(7, 6) 5 number: 393
(5, 5) -3 number: 394
(3, 3) 3 number: 395
(4, 4) -2 number: 396
(2, 2) 0 number: 397
(1, 1) 0 number: 398
(1, 1) 0 number: 399
(2, 2) 0 number: 400
(3, 3) -2 number: 401
(3, 3) -2 number: 402
(1, 1) 1 number: 403
(5, 4) -4 number: 404
(6, 6) -4 number: 405
(2, 2) 1 number: 406
(1, 1) 0 number: 407
(2, 2) -1 number: 408
(3, 3) -2 number: 409


In [42]:
name_bounds=[[k[0],k[1]] for k in l]

In [44]:
with open('census_upper_bounds.csv','w') as out:
        writer=csv.writer(out)
        for row in name_bounds:
            writer.writerow(row)

In [67]:
change_sequences=[[k[0],k[2]] for k in l]

In [70]:
with open('change_sequences.csv','w') as out:
        writer=csv.writer(out)
        for row in change_sequences:
            writer.writerow(row)